In [2]:
!pip install selenium
import pandas as pd
import requests
import selenium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries Imported')

Libraries Imported


### **Data Wrangling**

In [3]:
df= pd.read_csv('Neigh Rome.txt')

In [4]:
df.head()

,Neighborhood,Name
0,Q. I,Flaminio
1,Q. II,Parioli
2,Q. III,Pinciano
3,Q. IV,Salario
4,Q. V,Nomentano


In [5]:
!pip install geoPy

     |████████████████████████████████| 112kB 24.0MB/s eta 0:00:01


### **Convert addresses into Latitude and Longitude**

In [6]:
from geopy.geocoders import Nominatim
lat=[]
lng=[]
def getLatLng(row):
    geolocator = Nominatim(user_agent='foursquare')
    print(row[0]+', '+row[1])
    location = geolocator.geocode(row[0]+', '+row[1])
    if location != None:
        lat.append(location.latitude)
        lng.append(location.longitude)
    else:
        lat.append(None)
        lng.append(None)

In [7]:
df.apply(getLatLng, axis=1)
df.head()

Q. I,  Flaminio
Q. II,  Parioli
Q. III,  Pinciano
Q. IV,  Salario
Q. V,  Nomentano
Q. VI,  Tiburtino
Q. VII,  Prenestino-Labicano
Q. VIII,  Tuscolano
Q. IX,  Appio-Latino
Q. X,  Ostiense
Q. XI,  Portuense
Q. XII,  Gianicolense
Q. XIII,  Aurelio
Q. XIV,  Trionfale
Q. XV,  Della Vittoria
Q. XVI,  Monte Sacro
Q. XVII,  Trieste
Q. XVIII,  Tor di Quinto
Q. XIX,  Prenestino-Centocelle
Q. XX,  Ardeatino
Q. XXI,  Pietralata
Q. XXII,  Collatino
Q. XXIII,  Alessandrino
Q. XXIV,  Don Bosco
Q. XXV,  Appio Claudio
Q. XXVI,  Appio-Pignatelli
Q. XXVII,  Primavalle
Q. XXVIII,  Monte Sacro Alto
Q. XXIX,  Ponte Mammolo
Q. XXX,  San Basilio [it]
Q. XXXI,  Giuliano-Dalmata
Q. XXXII,  Europa
Q. XXXIII,  Lido di Ostia Ponente
Q. XXXIV,  Lido di Ostia Levante
Q. XXXV,  Lido di Castel Fusano


,Neighborhood,Name
0,Q. I,Flaminio
1,Q. II,Parioli
2,Q. III,Pinciano
3,Q. IV,Salario
4,Q. V,Nomentano


In [8]:
df['Latitude']=lat
df['Longitude']=lng
df.shape

(35, 4)

In [9]:
df.head()

,Neighborhood,Name,Latitude,Longitude
0,Q. I,Flaminio,-33.512521,-70.626609
1,Q. II,Parioli,41.925949,12.487320
2,Q. III,Pinciano,41.918781,12.485332
3,Q. IV,Salario,41.940398,12.508896
4,Q. V,Nomentano,41.917002,12.512918


In [10]:
df.dropna(axis=0, inplace=True)

In [11]:
df.head()

,Neighborhood,Name,Latitude,Longitude
0,Q. I,Flaminio,-33.512521,-70.626609
1,Q. II,Parioli,41.925949,12.487320
2,Q. III,Pinciano,41.918781,12.485332
3,Q. IV,Salario,41.940398,12.508896
4,Q. V,Nomentano,41.917002,12.512918


The latitude and the longitude of Q.I - Flaminio is clearly out of range, so we drop this line.

In [12]:
df.drop(df.index[0], inplace=True)
df.head()

,Neighborhood,Name,Latitude,Longitude
1,Q. II,Parioli,41.925949,12.487320
2,Q. III,Pinciano,41.918781,12.485332
3,Q. IV,Salario,41.940398,12.508896
4,Q. V,Nomentano,41.917002,12.512918
5,Q. VI,Tiburtino,41.896633,12.511238


### **Visuilize neighborhood of Rome using folium**

In [13]:
!pip install folium
import folium

In [14]:
hlat, hlng = 41.925949, 12.487320
rome_map = folium.Map([hlat, hlng], zoom_start=10)
rome_map

In [15]:
for label, lat, lng in zip(df['Neighborhood'], df['Latitude'], df['Longitude']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location=[lat, lng],
        radius=10,
        popup=label,
        colur='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        parse_html=False
    ).add_to(rome_map)
    
rome_map

### **Exploring Unique Venues**

In [16]:
CLIENT_ID = 'INGPTUHXFDDIOXXKK4CSCTRQESJAMMKX1EHSU5K2VVKDA2XX' # your Foursquare ID
CLIENT_SECRET = 'XUBH3Y3VTYVDBOIHVHUNP1YDCVTXJJ4WG0VFL42GWM2HJLCR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: INGPTUHXFDDIOXXKK4CSCTRQESJAMMKX1EHSU5K2VVKDA2XX
CLIENT_SECRET:XUBH3Y3VTYVDBOIHVHUNP1YDCVTXJJ4WG0VFL42GWM2HJLCR


In [17]:
Limit=100

In [18]:
def getNearbyRestaurants(names, latitudes, longitudes, radius=3000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&section=food&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            Limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
rome_restaurants = getNearbyRestaurants(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Q. II
Q. III
Q. IV
Q. V
Q. VI
Q. VII
Q. VIII
Q. IX
Q. X
Q. XI
Q. XII
Q. XIII
Q. XIV
Q. XV
Q. XVI
Q. XVII
Q. XVIII
Q. XIX
Q. XX
Q. XXI
Q. XXII
Q. XXIII
Q. XXIV
Q. XXV
Q. XXVI
Q. XXVII
Q. XXIX
Q. XXXII
Q. XXXV


In [22]:
print(rome_restaurants.shape)
len(rome_restaurants['Venue Latitude'].unique())

(2703, 7)


1156

In [23]:
rome_restaurants_unique = rome_restaurants.drop_duplicates(subset=['Venue Latitude', 'Venue Longitude'], keep='first')

In [81]:
rome_restaurants_unique[rome_restaurants_unique['Venue Category'].str.contains('Italian')]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3,Q. II,41.925949,12.487320,Metamorfosi Restaurant,41.924077,12.484641,Italian Restaurant
12,Q. II,41.925949,12.487320,Bucavino,41.917442,12.499237,Italian Restaurant
15,Q. II,41.925949,12.487320,Enoteca La Torre a Villa Laetitia,41.916261,12.469070,Italian Restaurant
18,Q. II,41.925949,12.487320,Larys,41.916530,12.499190,Italian Restaurant
19,Q. II,41.925949,12.487320,Ristorante Andrea,41.909045,12.489692,Italian Restaurant
...,...,...,...,...,...,...,...
2692,Q. XXXV,41.716500,12.323709,La Querida,41.733805,12.334911,Italian Restaurant
2696,Q. XXXV,41.716500,12.323709,cinghiale,41.731761,12.350988,Italian Restaurant
2697,Q. XXXV,41.716500,12.323709,Fraschetta Da Matteo,41.735674,12.345803,Italian Restaurant
2698,Q. XXXV,41.716500,12.323709,La Mariposa,41.709623,12.326375,Italian Restaurant


### **Visualize all the venues in Rome**

In [25]:
hlat, hlng = 41.925949, 12.487320
rest_map = folium.Map([hlat, hlng], zoom_start=10)
for label, lat, lng in zip(rome_restaurants_unique['Venue'], rome_restaurants_unique['Venue Latitude'], 
                           rome_restaurants_unique['Venue Longitude']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location=[lat, lng],
        radius=5,
        popup=label,
        colur='red',
        fill=True,
        fill_color='green',
        fill_opacity=0.6,
        parse_html=False
    ).add_to(rest_map)
from IPython.display import display
display(rest_map)

Let's check how many venues were returned for each neighborhood

In [26]:
rome_restaurants_unique.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Q. II,100,100,100,100,100,100
Q. III,17,17,17,17,17,17
Q. IV,62,62,62,62,62,62
Q. IX,50,50,50,50,50,50
Q. V,39,39,39,39,39,39
Q. VI,43,43,43,43,43,43
Q. VII,16,16,16,16,16,16
Q. VIII,73,73,73,73,73,73
Q. X,66,66,66,66,66,66


Let's find out how many unique categories can be curated from all the returned venues

In [27]:
print('There are {} uniques categories.'.format(len(rome_restaurants_unique['Venue Category'].unique())))

There are 68 uniques categories.


In [28]:
rome_onehot = pd.get_dummies(rome_restaurants_unique['Venue Category'])
rome_onehot.insert(loc=0, column='Neighborhood', value=rome_restaurants_unique['Neighborhood'])
rome_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,...,Soup Place,South American Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Thai Restaurant,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Q. II,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Q. II,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Q. II,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Q. II,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Q. II,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
rome_grouped = rome_onehot.groupby('Neighborhood').mean().reset_index()

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = rome_grouped['Neighborhood']

for ind in np.arange(rome_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(rome_grouped.iloc[ind, :], num_top_venues)

### **Top 10 Venues in each neighborhood**

In [82]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Q. II,Italian Restaurant,Restaurant,Pizza Place,Café,Sandwich Place,Diner,Japanese Restaurant,Trattoria/Osteria,Bistro,Bakery
1,1,Q. III,Italian Restaurant,Sandwich Place,Turkish Restaurant,Restaurant,Café,Korean Restaurant,Bistro,Vietnamese Restaurant,Ethiopian Restaurant,Falafel Restaurant
2,2,Q. IV,Italian Restaurant,Pizza Place,Café,Japanese Restaurant,Chinese Restaurant,Restaurant,Gastropub,Trattoria/Osteria,Steakhouse,Sandwich Place
3,2,Q. IX,Italian Restaurant,Pizza Place,Café,Restaurant,Roman Restaurant,Sandwich Place,Gastropub,Diner,Trattoria/Osteria,Fish & Chips Shop
4,2,Q. V,Pizza Place,Italian Restaurant,Café,Mediterranean Restaurant,Trattoria/Osteria,Gastropub,Seafood Restaurant,Sandwich Place,Salad Place,Roman Restaurant


### **Clustering Neighborhoods**

In [44]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

rome_grouped_clustering = rome_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(rome_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 2, 2, 2, 2, 2, 2, 2, 0], dtype=int32)

In [48]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

rome_merged = df

# merge rome_grouped with rome_data to add latitude/longitude for each neighborhood
rome_merged = rome_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
rome_merged # check the last columns!

,Neighborhood,Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Q. II,Parioli,41.925949,12.487320,2,Italian Restaurant,Restaurant,Pizza Place,Café,Sandwich Place,Diner,Japanese Restaurant,Trattoria/Osteria,Bistro,Bakery
2,Q. III,Pinciano,41.918781,12.485332,1,Italian Restaurant,Sandwich Place,Turkish Restaurant,Restaurant,Café,Korean Restaurant,Bistro,Vietnamese Restaurant,Ethiopian Restaurant,Falafel Restaurant
3,Q. IV,Salario,41.940398,12.508896,2,Italian Restaurant,Pizza Place,Café,Japanese Restaurant,Chinese Restaurant,Restaurant,Gastropub,Trattoria/Osteria,Steakhouse,Sandwich Place
4,Q. V,Nomentano,41.917002,12.512918,2,Pizza Place,Italian Restaurant,Café,Mediterranean Restaurant,Trattoria/Osteria,Gastropub,Seafood Restaurant,Sandwich Place,Salad Place,Roman Restaurant
5,Q. VI,Tiburtino,41.896633,12.511238,2,Italian Restaurant,Pizza Place,Restaurant,Bistro,Trattoria/Osteria,Sandwich Place,Greek Restaurant,Food Court,Soup Place,Bakery
6,Q. VII,Prenestino-Labicano,41.891549,12.516829,2,Italian Restaurant,Pizza Place,Bakery,Greek Restaurant,Gastropub,Restaurant,Steakhouse,Noodle House,Fish & Chips Shop,Ethiopian Restaurant
7,Q. VIII,Tuscolano,41.867638,12.539368,2,Pizza Place,Italian Restaurant,Café,Trattoria/Osteria,Restaurant,Steakhouse,Chinese Restaurant,Sandwich Place,Bakery,Bistro
8,Q. IX,Appio-Latino,41.878634,12.504255,2,Italian Restaurant,Pizza Place,Café,Restaurant,Roman Restaurant,Sandwich Place,Gastropub,Diner,Trattoria/Osteria,Fish & Chips Shop
9,Q. X,Ostiense,41.857283,12.477881,2,Pizza Place,Italian Restaurant,Café,Japanese Restaurant,Bakery,Restaurant,Mediterranean Restaurant,Roman Restaurant,Diner,Gastropub
10,Q. XI,Portuense,41.857503,12.449042,0,Italian Restaurant,Pizza Place,Café,Restaurant,Sandwich Place,Food,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Modern European Restaurant,Chinese Restaurant


In [46]:
rome_merged['Cluster Labels'] = rome_merged['Cluster Labels'].fillna(0)
rome_merged['Cluster Labels'] = rome_merged['Cluster Labels'].astype(int)
rome_merged.head()

,Neighborhood,Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Q. II,Parioli,41.925949,12.487320,2,Italian Restaurant,Restaurant,Pizza Place,Café,Sandwich Place,Diner,Japanese Restaurant,Trattoria/Osteria,Bistro,Bakery
2,Q. III,Pinciano,41.918781,12.485332,1,Italian Restaurant,Sandwich Place,Turkish Restaurant,Restaurant,Café,Korean Restaurant,Bistro,Vietnamese Restaurant,Ethiopian Restaurant,Falafel Restaurant
3,Q. IV,Salario,41.940398,12.508896,2,Italian Restaurant,Pizza Place,Café,Japanese Restaurant,Chinese Restaurant,Restaurant,Gastropub,Trattoria/Osteria,Steakhouse,Sandwich Place
4,Q. V,Nomentano,41.917002,12.512918,2,Pizza Place,Italian Restaurant,Café,Mediterranean Restaurant,Trattoria/Osteria,Gastropub,Seafood Restaurant,Sandwich Place,Salad Place,Roman Restaurant
5,Q. VI,Tiburtino,41.896633,12.511238,2,Italian Restaurant,Pizza Place,Restaurant,Bistro,Trattoria/Osteria,Sandwich Place,Greek Restaurant,Food Court,Soup Place,Bakery


In [36]:
# create map
map_clusters = folium.Map(location=[hlat, hlng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(rome_merged['Latitude'], rome_merged['Longitude'], rome_merged['Neighborhood'], rome_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### **Examining the Clusters - Results**

#### **Cluster 1 (RED)**

In [83]:
required_column_indices = [5,6]
required_column = [list(rome_merged.columns.values)[i] for i in required_column_indices]
required_column_indices = [5,6]

In [84]:
cluster_1 = rome_merged.loc[rome_merged['Cluster Labels'] == 0, rome_merged.columns[[0] + list(range(5, rome_merged.shape[1]))]]
cluster_1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Q. XI,Italian Restaurant,Pizza Place,Café,Restaurant,Sandwich Place,Food,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Modern European Restaurant,Chinese Restaurant
15,Q. XVI,Italian Restaurant,Café,Pizza Place,Sushi Restaurant,Restaurant,Diner,Japanese Restaurant,Mexican Restaurant,Thai Restaurant,Gastropub
17,Q. XVIII,Italian Restaurant,Café,Pizza Place,Restaurant,Bakery,Sandwich Place,Seafood Restaurant,Steakhouse,Asian Restaurant,Gastropub
19,Q. XX,Café,Italian Restaurant,Pizza Place,Restaurant,Japanese Restaurant,Bistro,Sandwich Place,Moroccan Restaurant,Fast Food Restaurant,Chinese Restaurant
20,Q. XXI,Pizza Place,Café,Italian Restaurant,Japanese Restaurant,Chinese Restaurant,Restaurant,Steakhouse,Seafood Restaurant,Sandwich Place,African Restaurant
24,Q. XXV,Café,Pizza Place,Italian Restaurant,Restaurant,Bakery,Noodle House,Middle Eastern Restaurant,Fried Chicken Joint,Trattoria/Osteria,Chinese Restaurant
26,Q. XXVII,Café,Pizza Place,Italian Restaurant,Chinese Restaurant,Restaurant,Fast Food Restaurant,Bakery,Sandwich Place,Vegetarian / Vegan Restaurant,Peruvian Restaurant
28,Q. XXIX,Café,Italian Restaurant,Pizza Place,Seafood Restaurant,Food Court,Restaurant,Fast Food Restaurant,Burger Joint,Diner,Bistro
31,Q. XXXII,Café,Italian Restaurant,Pizza Place,Restaurant,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Sushi Restaurant,Steakhouse,Bakery


In [85]:
for col in required_column:
    print(cluster_1[col].value_counts(ascending = False))
    print("---------------------------------------------")

Café                  5
Italian Restaurant    3
Pizza Place           1
Name: 1st Most Common Venue, dtype: int64
---------------------------------------------
Pizza Place           3
Café                  3
Italian Restaurant    3
Name: 2nd Most Common Venue, dtype: int64
---------------------------------------------


#### **Cluster 2 (Violet)**

In [86]:
cluster_2=rome_merged.loc[rome_merged['Cluster Labels'] == 1, rome_merged.columns[[0] + list(range(5, rome_merged.shape[1]))]]
cluster_2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Q. III,Italian Restaurant,Sandwich Place,Turkish Restaurant,Restaurant,Café,Korean Restaurant,Bistro,Vietnamese Restaurant,Ethiopian Restaurant,Falafel Restaurant
12,Q. XIII,Italian Restaurant,Sandwich Place,Restaurant,Bakery,Café,Falafel Restaurant,Trattoria/Osteria,Pizza Place,Romagna Restaurant,Fondue Restaurant
13,Q. XIV,Italian Restaurant,Café,Sandwich Place,Pizza Place,Diner,Vegetarian / Vegan Restaurant,Restaurant,Chinese Restaurant,Salad Place,Vietnamese Restaurant
25,Q. XXVI,Italian Restaurant,Sandwich Place,Gastropub,Café,Vietnamese Restaurant,Fondue Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop


In [87]:
for col in required_column:
    print(cluster_2[col].value_counts(ascending = False))
    print("---------------------------------------------")

Italian Restaurant    4
Name: 1st Most Common Venue, dtype: int64
---------------------------------------------
Sandwich Place    3
Café              1
Name: 2nd Most Common Venue, dtype: int64
---------------------------------------------


#### **Cluster 3 (Light blue)**

In [88]:
cluster_3=rome_merged.loc[rome_merged['Cluster Labels'] == 2, rome_merged.columns[[0] + list(range(5, rome_merged.shape[1]))]]
cluster_3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Q. II,Italian Restaurant,Restaurant,Pizza Place,Café,Sandwich Place,Diner,Japanese Restaurant,Trattoria/Osteria,Bistro,Bakery
3,Q. IV,Italian Restaurant,Pizza Place,Café,Japanese Restaurant,Chinese Restaurant,Restaurant,Gastropub,Trattoria/Osteria,Steakhouse,Sandwich Place
4,Q. V,Pizza Place,Italian Restaurant,Café,Mediterranean Restaurant,Trattoria/Osteria,Gastropub,Seafood Restaurant,Sandwich Place,Salad Place,Roman Restaurant
5,Q. VI,Italian Restaurant,Pizza Place,Restaurant,Bistro,Trattoria/Osteria,Sandwich Place,Greek Restaurant,Food Court,Soup Place,Bakery
6,Q. VII,Italian Restaurant,Pizza Place,Bakery,Greek Restaurant,Gastropub,Restaurant,Steakhouse,Noodle House,Fish & Chips Shop,Ethiopian Restaurant
7,Q. VIII,Pizza Place,Italian Restaurant,Café,Trattoria/Osteria,Restaurant,Steakhouse,Chinese Restaurant,Sandwich Place,Bakery,Bistro
8,Q. IX,Italian Restaurant,Pizza Place,Café,Restaurant,Roman Restaurant,Sandwich Place,Gastropub,Diner,Trattoria/Osteria,Fish & Chips Shop
9,Q. X,Pizza Place,Italian Restaurant,Café,Japanese Restaurant,Bakery,Restaurant,Mediterranean Restaurant,Roman Restaurant,Diner,Gastropub
11,Q. XII,Italian Restaurant,Pizza Place,Café,Restaurant,Chinese Restaurant,Burger Joint,Steakhouse,Breakfast Spot,Donut Shop,Diner
14,Q. XV,Italian Restaurant,Pizza Place,Restaurant,Japanese Restaurant,Sandwich Place,Mexican Restaurant,Creperie,Café,Burger Joint,Roman Restaurant


In [89]:
for col in required_column:
    print(cluster_3[col].value_counts(ascending = False))
    print("---------------------------------------------")

Italian Restaurant    9
Pizza Place           4
Name: 1st Most Common Venue, dtype: int64
---------------------------------------------
Pizza Place           7
Italian Restaurant    3
Seafood Restaurant    1
Café                  1
Restaurant            1
Name: 2nd Most Common Venue, dtype: int64
---------------------------------------------


#### **Cluster 4 (Green)**

In [90]:
cluster_4 =rome_merged.loc[rome_merged['Cluster Labels'] == 3, rome_merged.columns[[0] + list(range(5, rome_merged.shape[1]))]]
cluster_4

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Q. XXXV,Italian Restaurant,Seafood Restaurant,Mediterranean Restaurant,Vietnamese Restaurant,Fondue Restaurant,Donut Shop,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop


#### **Cluster 5 (Orange)**

In [91]:
cluster_5 = rome_merged.loc[rome_merged['Cluster Labels'] == 4, rome_merged.columns[[0] + list(range(5, rome_merged.shape[1]))]]
cluster_5

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Q. XXII,Pizza Place,Restaurant,Sushi Restaurant,Spanish Restaurant,Kebab Restaurant,Fast Food Restaurant,Deli / Bodega,Diner,Donut Shop,Ethiopian Restaurant
22,Q. XXIII,Pizza Place,Café,Fast Food Restaurant,Roman Restaurant,Vietnamese Restaurant,Donut Shop,Ethiopian Restaurant,Falafel Restaurant,Fish & Chips Shop,Fondue Restaurant
